## Using cloud-top height and visible reflectance from GOES-16 satellite images to infer cloud area fraction $a_c$: calculation and data output

### Python packages

In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import os
import glob
import re

### Functions

In [2]:
def extract_yyyydoy(fpath):
    """
    Return integer YYYYDOY if found in the basename of fpath; else None.
    """
    fname = os.path.basename(fpath)
    m = doy_pattern.search(fname)
    return int(m.group(1)) if m else None

def extract_hhmm(fpath):
    """
    Return 4-digit HHMM if found (as the group after YYYYDOY); else "0000".
    """
    fname = os.path.basename(fpath)
    m = time_pattern.search(fname)
    return m.group(2) if m else "0000"

def preprocess(ds):
    """
    If 'mc_calibhed_size' appears in ds.dims, drop it;
    otherwise return ds unchanged.
    """
    if 'mc_calibhed_size' in ds.dims:
        return ds.drop_dims('mc_calibhed_size', errors='ignore')
    return ds

### Loading data

In [3]:
# Time range
start_doy = 2020015  # 015 January 15, 2020
end_doy   = 2020050  # 050 February 19, 2020

# Base directory
base_dir = '/home/huangany/data/KloudatA/GOES-16/2020'

# Output directory
output_dir = os.path.join(base_dir, 'rfv_bin_4km') # Full domain
#output_dir = os.path.join(base_dir, 'rfv_bin_NW') # NW domain
#output_dir = os.path.join(base_dir, 'rfv_bin_NE') # NE domain
os.makedirs(output_dir, exist_ok=True)

# Regex to extract the 7-digit YYYYDOY and 4-digit HHMM from each filename
doy_pattern  = re.compile(r'(\d{7})\.\d{4}')
time_pattern = re.compile(r'\.(\d{7})\.(\d{4})\.')

In [4]:
# Gather & filter all .NC files by DOY range
all_files = sorted(
    glob.glob(os.path.join(base_dir, '01', '**', '*.NC')) +
    glob.glob(os.path.join(base_dir, '02', '**', '*.NC'))
)
filtered_files = [
    f for f in all_files
    if (extract_yyyydoy(f) is not None) and (start_doy <= extract_yyyydoy(f) <= end_doy)
]

# Define cloud-top-height bins (0.5 km → 20 km, step 0.01 km)
# cth_bins = np.arange(0.5, 20.01, 0.01)  # edges: 0.5, 0.51, …, 20.0
cth_bins = np.arange(0.5, 4.01, 0.01)  # Only interested low-clouds
n_bins = len(cth_bins)

In [5]:
# Define box limits (longitude and latitude): Atomic Field Campaign region
lon_min, lon_max = -60, -49  # W is negative
lat_min, lat_max = 12.5, 16  # N is positive

### Loop over each filtered file and compute & save

In [6]:
for file_path in filtered_files:
    # a) Extract YYYYDOY and HHMM for filenames & summary
    yyyydoy = extract_yyyydoy(file_path)
    hhmm    = extract_hhmm(file_path)

    # b) Open the dataset and drop unwanted dims if present
    ds = xr.open_dataset(file_path)
    ds = preprocess(ds)

    # c) Compute full-domain average solar zenith angle (SZA)
    sza = ds['solar_zenith_angle'].values    # shape: (ny, nx)
    avg_sza = float(np.nanmean(sza))          # scalar

    # d) If avg_sza > 90, write summary line and skip all other output
    if avg_sza > 90.0:
        #f_summary.write(f"{yyyydoy}.{hhmm},{avg_sza:.1f},{valid_fraction:.1f}\n")
        ds.close()
        continue
    else:
        print(file_path)

    # e) Build region mask (X<901, Y<901) once, since needed for valid_fraction either way
    rfv = ds['reflectance_vis'].values       # shape: (ny, nx)
    cth = ds['cloud_top_height'].values      # shape: (ny, nx)
    
    #ny, nx = rfv.shape
    #Y, X = np.indices((ny, nx))
    #region_mask = (X < 901) & (Y < 901) # Full domain
    #region_mask = (X < 450) & (Y < 450) # NW domain
    #region_mask = (X > 450) & (Y < 450) # NE domain
    # Boolean mask where lat-lon is inside the desired box
    region_mask = (
        (ds.longitude >= lon_min) & (ds.longitude <= lon_max) &
        (ds.latitude >= lat_min) & (ds.latitude <= lat_max)
    )

    # f) Mask cth<0.5 and cth>4 km → 0, build valid_mask, compute valid_fraction
    cth = np.where(np.isnan(cth), 0, cth)
    cth = np.where(cth < 0.5, 0, cth)
    cth = np.where(cth > 4.0, np.nan, cth) # Mask out high clouds (higher than 4 km)
    cth = np.where(np.isnan(rfv), np.nan, cth)
    rfv = np.where(rfv > 1, 1, rfv)
    valid_mask = (~np.isnan(rfv)) & (~np.isnan(cth)) & region_mask
    total_mask = (~np.isnan(cth)) & region_mask

    # total_pixels   = np.sum(region_mask).values
    total_pixels   = np.sum(total_mask).values # Excluding high clouds
    valid_pixels   = np.sum(valid_mask).values
    valid_fraction = 100.0 * valid_pixels / total_pixels
    # print(f'Valid fraction: {valid_fraction} %')

    # g) Otherwise (avg_sza ≤ 90), do the full binning workflow:

    #    1) Mask out cth<0.5 km (already done above), extract valid pixels
    rfv_flat = rfv[valid_mask]
    cth_flat = cth[valid_mask]
    
    #    2) Digitize cth into bins
    inds = np.digitize(cth_flat, cth_bins) - 1
    valid_inds = (inds >= 0) & (inds < n_bins)
    inds = inds[valid_inds]
    rfv_vals = rfv_flat[valid_inds]

    #    3) Compute bin sums & counts
    bin_sums   = np.bincount(inds, weights=rfv_vals, minlength=n_bins)
    bin_counts = np.bincount(inds, minlength=n_bins)

    #    4) Compute rfv_avg and rfv_nrm
    # rfv_avg = bin_sums / bin_counts
    rfv_nrm = bin_sums / total_pixels

    #    5) Compute backward-accumulated reflectance
    rfv_acc = np.flip(np.nancumsum(np.flip(rfv_nrm, axis=0), axis=0), axis=0)

    # h) Build an xarray Dataset to save out to NetCDF
    ds_out = xr.Dataset(
        {
            # 1D variable of bin edges
            "cth_bins": (("cth_bin",), cth_bins),

            # Binned reflectance variables
            # "rfv_avg": (("cth_bin",), rfv_avg),
            "rfv_sum": (("cth_bin",),   bin_sums),
            "bin_count": (("cth_bin",), bin_counts),
            "rfv_nrm": (("cth_bin",),   rfv_nrm),
            "rfv_acc": (("cth_bin",),   rfv_acc),

            # Scalar metadata
            "valid_fraction": valid_fraction,
            "total_pixels":   total_pixels,
            "valid_pixels":   valid_pixels,
            "avg_sza":        avg_sza,
        },
        coords={
            "cth_bin": cth_bins
        },
        attrs={
            "source_file": os.path.basename(file_path),
            "YYYYDOY":     str(yyyydoy),
            "HHMM":        hhmm
        }
    )

    # i) Save the Dataset to a NetCDF file named by YYYYDOY_HHMM
    out_fname = os.path.join(output_dir, f"goes16_binned_low4km_{yyyydoy}_{hhmm}.nc") # Full domain
    #out_fname = os.path.join(output_dir, f"goes16_binned_NW_{yyyydoy}_{hhmm}.nc") # NW domain
    #out_fname = os.path.join(output_dir, f"goes16_binned_NE_{yyyydoy}_{hhmm}.nc") # NE domain
    ds_out.to_netcdf(out_fname)

    # k) Close both Datasets before moving on
    ds_out.close()
    ds.close()


/home/huangany/data/KloudatA/GOES-16/2020/01/15/G16V04.0.ATOMIC.2020015.1040.PX.02K.NC
/home/huangany/data/KloudatA/GOES-16/2020/01/15/G16V04.0.ATOMIC.2020015.1100.PX.02K.NC
/home/huangany/data/KloudatA/GOES-16/2020/01/15/G16V04.0.ATOMIC.2020015.1120.PX.02K.NC
/home/huangany/data/KloudatA/GOES-16/2020/01/15/G16V04.0.ATOMIC.2020015.1140.PX.02K.NC
/home/huangany/data/KloudatA/GOES-16/2020/01/15/G16V04.0.ATOMIC.2020015.1200.PX.02K.NC
/home/huangany/data/KloudatA/GOES-16/2020/01/15/G16V04.0.ATOMIC.2020015.1220.PX.02K.NC
/home/huangany/data/KloudatA/GOES-16/2020/01/15/G16V04.0.ATOMIC.2020015.1240.PX.02K.NC
/home/huangany/data/KloudatA/GOES-16/2020/01/15/G16V04.0.ATOMIC.2020015.1300.PX.02K.NC
/home/huangany/data/KloudatA/GOES-16/2020/01/15/G16V04.0.ATOMIC.2020015.1320.PX.02K.NC
/home/huangany/data/KloudatA/GOES-16/2020/01/15/G16V04.0.ATOMIC.2020015.1340.PX.02K.NC
/home/huangany/data/KloudatA/GOES-16/2020/01/15/G16V04.0.ATOMIC.2020015.1400.PX.02K.NC
/home/huangany/data/KloudatA/GOES-16/2020/0